In [3]:
# 1) Design model (input size, output size, forward pass)
# 2) Construct loss and optimizer
# 3) Training loop
#    - forward pass: compute prediction and loss
#    - backward pass: gradients
#    - update weights

In [2]:
import torch
import torchvision # some builtin datasets
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import math

In [8]:
# 0) Prepare data
class WineDataset(Dataset):
    def __init__(self):
        # data loading
        xy = np.loadtxt("./data/wine.csv", delimiter=",", dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(xy[:, 1:]) #skip the first column
        self.y = torch.from_numpy(xy[:, 0])
        self.n_samples = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index,:], self.y[index]

    def __len__(self):
        return self.n_samples

dataset = WineDataset()
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers=2)

first_data_x, first_data_y = dataset[0]
print(first_data_x)
print(first_data_y)

data_iter = iter(dataloader)
X, y = data_iter.next()
print(X)
print(y)

# dummy training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)

for epoch in range(num_epochs):
    for i, (inputs, lables) in enumerate(dataloader):
        # forward
        # backward
        # update
        if (i+1) % 5 == 0:
            print(f'epoch {epoch + 1}/{num_epochs}, step {i+1}/{n_iterations}, inputs {inputs.shape}')




tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
tensor(1.)
tensor([[1.2510e+01, 1.7300e+00, 1.9800e+00, 2.0500e+01, 8.5000e+01, 2.2000e+00,
         1.9200e+00, 3.2000e-01, 1.4800e+00, 2.9400e+00, 1.0400e+00, 3.5700e+00,
         6.7200e+02],
        [1.3750e+01, 1.7300e+00, 2.4100e+00, 1.6000e+01, 8.9000e+01, 2.6000e+00,
         2.7600e+00, 2.9000e-01, 1.8100e+00, 5.6000e+00, 1.1500e+00, 2.9000e+00,
         1.3200e+03],
        [1.3200e+01, 1.7800e+00, 2.1400e+00, 1.1200e+01, 1.0000e+02, 2.6500e+00,
         2.7600e+00, 2.6000e-01, 1.2800e+00, 4.3800e+00, 1.0500e+00, 3.4000e+00,
         1.0500e+03],
        [1.1450e+01, 2.4000e+00, 2.4200e+00, 2.0000e+01, 9.6000e+01, 2.9000e+00,
         2.7900e+00, 3.2000e-01, 1.8300e+00, 3.2500e+00, 8.0000e-01, 3.3900e+00,
         6.2500e+02]])
tensor([2., 1., 1., 2.])
178 45
epoch 1/2, step 5/45, inputs torch.Size

In [13]:
# model: y = w0x0 + w1x1 + b, sigmoid at the end
input_size = n_features
output_size = 1 # binary

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define layers
        self.lin = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return torch.sigmoid(self.lin(x))

model = LinearRegression(input_size, output_size)

#loss criterion and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [16]:
# Training loop
n_iters = 200

for epoch in range(n_iters):
    # forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # backward pass
    loss.backward()

    # update weights
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch {epoch+1}: loss = {loss: 0.8f}')

# Evaluation
with torch.no_grad():
    y_eva = model(X_test)
    y_eva_cls = y_eva.round()
    acc = y_eva_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'acc = {acc:.4f}')

epoch 10: loss =  0.17208649
epoch 20: loss =  0.16849808
epoch 30: loss =  0.16514289
epoch 40: loss =  0.16199709
epoch 50: loss =  0.15904003
epoch 60: loss =  0.15625384
epoch 70: loss =  0.15362300
epoch 80: loss =  0.15113385
epoch 90: loss =  0.14877442
epoch 100: loss =  0.14653404
epoch 110: loss =  0.14440328
epoch 120: loss =  0.14237373
epoch 130: loss =  0.14043783
epoch 140: loss =  0.13858879
epoch 150: loss =  0.13682044
epoch 160: loss =  0.13512729
epoch 170: loss =  0.13350432
epoch 180: loss =  0.13194691
epoch 190: loss =  0.13045090
epoch 200: loss =  0.12901251
acc = 0.9298
